In [1]:
import numpy as np
from PIL import Image
import io
from pathlib import Path
import os
import torch
import matplotlib.pyplot as plt
import random
from peft import LoraConfig, get_peft_model, TaskType

In [2]:
# On utilise %pip pour cibler le bon environnement Python
# On ajoute --user pour éviter l'erreur "Accès refusé"

print("⏳ Installation de PyTorch (version CUDA)...")
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu122 --user

print("⏳ Installation des outils d'image...")
%pip install opencv-python pillow numpy matplotlib tqdm --user

print("⏳ Installation de Transformers et Hugging Face...")
%pip install transformers huggingface-hub peft accelerate --user

print("✅ Installation terminée. VEUILLEZ REDÉMARRER LE KERNEL.")

⏳ Installation de PyTorch (version CUDA)...
Looking in indexes: https://download.pytorch.org/whl/cu122
Note: you may need to restart the kernel to use updated packages.
⏳ Installation des outils d'image...


ERROR: Could not find a version that satisfies the requirement torchvision (from versions: none)
ERROR: No matching distribution found for torchvision


Note: you may need to restart the kernel to use updated packages.
⏳ Installation de Transformers et Hugging Face...
Note: you may need to restart the kernel to use updated packages.
✅ Installation terminée. VEUILLEZ REDÉMARRER LE KERNEL.


In [3]:
from transformers import AutoImageProcessor, AutoModelForDepthEstimation
print("✅ Import réussi !")

✅ Import réussi !


In [4]:
import numpy as np

print("=" * 50)
print("DETAILED CHANNEL ANALYSIS")
print("=" * 50)

# Check each channel separately
for i in range(3):
    channel = depth_data[:, :, i]
    valid_count = (~np.isnan(channel)).sum()
    nan_count = np.isnan(channel).sum()

    print(f"\nChannel {i}:")
    print(f"  Valid values: {valid_count} ({valid_count / channel.size * 100:.2f}%)")
    print(f"  NaN values: {nan_count} ({nan_count / channel.size * 100:.2f}%)")

    if valid_count > 0:
        print(f"  Min (ignoring NaN): {np.nanmin(channel):.2f}")
        print(f"  Max (ignoring NaN): {np.nanmax(channel):.2f}")
        print(f"  Mean (ignoring NaN): {np.nanmean(channel):.2f}")

# Overall statistics
total_pixels = depth_data.shape[0] * depth_data.shape[1]
total_values = depth_data.size
valid_pixels_count = (~np.isnan(depth_data)).any(axis=2).sum()

print(f"\n{'=' * 50}")
print("OVERALL STATISTICS")
print("=" * 50)
print(f"Total pixels: {total_pixels}")
print(f"Pixels with at least one valid value: {valid_pixels_count} ({valid_pixels_count/total_pixels*100:.1f}%)")
print(f"Total values (across 3 channels): {total_values}")
print(f"Total valid values: {(~np.isnan(depth_data)).sum()}")

# Try to find and display some valid values if they exist
print(f"\n{'=' * 50}")
print("SEARCHING FOR VALID DATA...")
print("=" * 50)

valid_mask = ~np.isnan(depth_data)
if valid_mask.any():
    # Find pixels with valid data
    valid_pixels = np.any(valid_mask, axis=2)
    valid_coords = np.argwhere(valid_pixels)

    if len(valid_coords) > 0:
        print(f"Found {len(valid_coords)} pixels with valid data!")
        print(f"\nFirst 3 valid pixels:")
        for idx in valid_coords[:3]:
            y, x = idx
            print(f"  Position ({y}, {x}): {depth_data[y, x]}")
else:
    print("⚠️  WARNING: NO VALID DATA FOUND IN ENTIRE FILE!")
    print("\nPossible issues:")
    print("  1. File is corrupted")
    print("  2. Wrong file exported from Zivid")
    print("  3. Data needs special loading procedure")

DETAILED CHANNEL ANALYSIS


NameError: name 'depth_data' is not defined

Preparation de l'env


In [5]:
def load_pretrained_model(model_name="depth-anything/Depth-Anything-V2-Small-hf"):
    """
    Charge le modèle Depth Anything V2 pré-entraîné et son processeur d'images
    """
    print(f"📥 Chargement du modèle: {model_name}")

    image_processor = AutoImageProcessor.from_pretrained(model_name)
    base_model = AutoModelForDepthEstimation.from_pretrained(model_name)

    print(f"✅ Modèle chargé avec succès")
    return base_model, image_processor

In [6]:
base_model, image_processor = load_pretrained_model()

📥 Chargement du modèle: depth-anything/Depth-Anything-V2-Small-hf


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


✅ Modèle chargé avec succès


In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
base_model = base_model.to(device)

In [8]:
# Count parameters
total_params = sum(p.numel() for p in base_model.parameters())
trainable_params = sum(p.numel() for p in base_model.parameters() if p.requires_grad)

print(f"📊 Total parameters: {total_params:,}")
print(f"📊 Trainable parameters: {trainable_params:,}")

📊 Total parameters: 24,785,089
📊 Trainable parameters: 24,785,089


In [9]:
# ===================================================
# INSPECT MODEL ARCHITECTURE
# ===================================================
print("🔍 Inspecting model architecture...")
print("\n" + "="*60)

# Print all module names
print("MODEL STRUCTURE:")
for name, module in base_model.named_modules():
    print(f"  {name}: {module.__class__.__name__}")

print("\n" + "="*60)
print("TRAINABLE LAYERS (before LoRA):")
for name, param in base_model.named_parameters():
    if param.requires_grad:
        print(f"  {name}: {param.shape}")

🔍 Inspecting model architecture...

MODEL STRUCTURE:
  : DepthAnythingForDepthEstimation
  backbone: Dinov2Backbone
  backbone.embeddings: Dinov2Embeddings
  backbone.embeddings.patch_embeddings: Dinov2PatchEmbeddings
  backbone.embeddings.patch_embeddings.projection: Conv2d
  backbone.embeddings.dropout: Dropout
  backbone.encoder: Dinov2Encoder
  backbone.encoder.layer: ModuleList
  backbone.encoder.layer.0: Dinov2Layer
  backbone.encoder.layer.0.norm1: LayerNorm
  backbone.encoder.layer.0.attention: Dinov2Attention
  backbone.encoder.layer.0.attention.attention: Dinov2SelfAttention
  backbone.encoder.layer.0.attention.attention.query: Linear
  backbone.encoder.layer.0.attention.attention.key: Linear
  backbone.encoder.layer.0.attention.attention.value: Linear
  backbone.encoder.layer.0.attention.output: Dinov2SelfOutput
  backbone.encoder.layer.0.attention.output.dense: Linear
  backbone.encoder.layer.0.attention.output.dropout: Dropout
  backbone.encoder.layer.0.layer_scale1: Dinov

PREPARATION  des données

In [10]:

path_images = r"P:\images"
path_depths = r"P:\depth"

In [11]:
print(f"📂 Dossier Images visé : {path_images}")
print(f"📂 Dossier Depth visé  : {path_depths}")

# Vérification que les dossiers existent
if os.path.exists(path_images) and os.path.exists(path_depths):
    print("\n✅ Les dossiers existent !")
    
    # Compter les fichiers pour être sûr
    nb_img = len([f for f in os.listdir(path_images) if f.endswith(('.png', '.jpg'))])
    nb_npy = len([f for f in os.listdir(path_depths) if f.endswith('.npy')])
    
    print(f"   -> Trouvé {nb_img} images")
    print(f"   -> Trouvé {nb_npy} fichiers XYZ (.npy)")
else:
    print("\n❌ ERREUR : Un des chemins est introuvable.")
    print("   Vérifie l'orthographe ou copie le chemin absolu.")

📂 Dossier Images visé : P:\images
📂 Dossier Depth visé  : P:\depth

✅ Les dossiers existent !
   -> Trouvé 58 images
   -> Trouvé 58 fichiers XYZ (.npy)


In [12]:
# 2. COMPTAGE ET LISTING
# ==========================================
print("🔍 Démarrage de la vérification...")

# On liste et on trie pour garantir l'ordre
# On ne garde que les fichiers (pas les dossiers cachés)
files_img = sorted([f for f in os.listdir(path_images) if f.endswith(('.png', '.jpg'))])
files_npy = sorted([f for f in os.listdir(path_depths) if f.endswith('.npy')])

num_img = len(files_img)
num_npy = len(files_npy)

print(f"📊 Bilan comptable :")
print(f"   - Images (.png/.jpg) : {num_img}")
print(f"   - Depths (.npy)      : {num_npy}")

🔍 Démarrage de la vérification...
📊 Bilan comptable :
   - Images (.png/.jpg) : 58
   - Depths (.npy)      : 58


In [13]:
# ==========================================
# 3. VÉRIFICATION DE L'ALIGNEMENT
# ==========================================
if num_img != num_npy:
    print(f"❌ ATTENTION : Le nombre de fichiers ne correspond pas ! (Différence : {abs(num_img - num_npy)})")
else:
    print("✅ Le nombre de fichiers est identique.")

# Vérification des correspondances de noms (optionnel mais recommandé)
# On vérifie si image_01_color.png a bien son équivalent image_01_rawDepth.npy
mismatches = []
for img_name in files_img:
    # On recrée le nom attendu du fichier depth (selon la logique Zivid vue dans ton notebook)
    # Ex: "scan_color.png" -> "scan_rawDepth.npy"
    expected_depth_name = img_name.replace("_color.png", "_rawDepth.npy").replace(".png", ".npy")
    
    # Si tes fichiers n'ont pas "_color", ajuste cette ligne (ex: replace(".png", ".npy"))
    
    if expected_depth_name not in files_npy:
        mismatches.append((img_name, expected_depth_name))

if len(mismatches) > 0:
    print(f"⚠️ ATTENTION : {len(mismatches)} paires ne correspondent pas au niveau du nom !")
    print(f"   Exemple de manquant : {mismatches[0]}")
else:
    print("✅ Tous les noms de fichiers semblent correspondre (logique color <-> rawDepth).")

✅ Le nombre de fichiers est identique.
✅ Tous les noms de fichiers semblent correspondre (logique color <-> rawDepth).


In [14]:
# ==========================================
# 4. TEST D'EXTRACTION DU Z (Le fameux Z)
# ==========================================

if num_npy > 0:
    print("\n🧪 Test d'extraction du canal Z sur un fichier aléatoire...")
    
    # Prendre un fichier au hasard
    random_npy = random.choice(files_npy)
    full_path = os.path.join(path_depths, random_npy)
    
    try:
        # Chargement
        data = np.load(full_path) # Shape (H, W, 3) normalement
        
        print(f"   Fichier : {random_npy}")
        print(f"   Shape originale : {data.shape}")
        
        if len(data.shape) == 3 and data.shape[2] == 3:
            # EXTRACTION DU Z (Canal index 2)
            Z_channel = data[:, :, 2]
            
            # Vérif statistiques
            print(f"   ✅ Extraction Z réussie ! Shape finale : {Z_channel.shape}")
            print(f"   Val Min (avec NaN) : {np.nanmin(Z_channel)}")
            print(f"   Val Max (avec NaN) : {np.nanmax(Z_channel)}")
            
            # Vérification des NaN
            nan_count = np.isnan(Z_channel).sum()
            print(f"   Nombre de pixels NaN (vides) : {nan_count} ({(nan_count/Z_channel.size)*100:.1f}%)")
        else:
            print("❌ Erreur : Le fichier .npy n'a pas 3 canaux (XYZ). Vérifie le format.")
            
    except Exception as e:
        print(f"❌ Erreur lors du chargement : {e}")


🧪 Test d'extraction du canal Z sur un fichier aléatoire...
   Fichier : 22-10-25-16-37-38_Zivid_acquisition_rawDepth.npy
   Shape originale : (1200, 1944, 3)
   ✅ Extraction Z réussie ! Shape finale : (1200, 1944)
   Val Min (avec NaN) : 287.8849792480469
   Val Max (avec NaN) : 2807.615478515625
   Nombre de pixels NaN (vides) : 665725 (28.5%)


In [15]:
filename = "21-12-03-19-05-46_Zivid_acquisition_rawDepth.npy"
full_path = os.path.join(path_depths, filename)

# --- EXTRACTION ---
# 1. Chargement de la donnée (1200, 1944, 3)
data_xyz = np.load(full_path)

# 2. Extraction de la carte de profondeur Z (1200, 1944)
Z_channel = data_xyz[:, :, 2] 

# 3. Traitement des NaN (pour s'assurer qu'on peut lire)
Z_clean = np.nan_to_num(Z_channel, nan=0.0)

# --- LECTURE DE LA VALEUR D'UN PIXEL ---
pixel_row = 600
pixel_col = 972

# Accéder à la valeur Z du pixel (600, 972)
profondeur_pixel = Z_clean[pixel_row, pixel_col]

print(f"Le fichier chargé est de forme : {Z_channel.shape}")
print(f"La valeur de Profondeur (Z) au pixel [{pixel_row}, {pixel_col}] est :")
print(f"-> **{profondeur_pixel:.2f} mm**")

Le fichier chargé est de forme : (1200, 1944)
La valeur de Profondeur (Z) au pixel [600, 972] est :
-> **1365.02 mm**


Nous avons choisi de traiter les données (extraire le canal Z et remplacer les NaN par 0.0) dans la fonction __getitem__ de la classe Dataset (c'est-à-dire pendant l'entraînement) au lieu de le faire une seule fois en amont.1. Efficacité et Économie de Stockage (Le Plus Important) Votre jeu de données est lourd, et l'objectif est d'éviter de créer des doublons :Données Brutes : Chaque fichier .npy stocke 3 canaux (X, Y, Z).Données Cible : Nous n'avons besoin que d'1 canal (Z).Si nous avions extrait Z et créé un nouveau fichier .npy pour chaque image :Vous auriez doublé le nombre de fichiers sur votre disque dur (un dossier images, un dossier depth_xyz, et un nouveau dossier depth_z_seul).Bien que le fichier de profondeur Z soit plus petit que le fichier XYZ, vous auriez passé un temps considérable à générer et stocker des milliers de nouveaux fichiers.En faisant l'opération à la volée, le processus est : Lire le fichier original (XYZ) $\rightarrow$ Extraire Z en mémoire RAM $\rightarrow$ Transférer vers le GPU. Aucune duplication n'est nécessaire sur le disque.

Configuration de LoRA


In [16]:

# --- 3. DÉFINITION DE LA CONFIGURATION LoRA ---

# Pour les Vision Transformers, nous ciblons les couches d'attention (Query, Key, Value)
lora_config = LoraConfig(
    r=16,                                    # Le rang de la matrice LoRA (ajustement de la capacité d'apprentissage)
    lora_alpha=32,                           # Facteur de mise à l'échelle (généralement 2 * r)
    target_modules=["query", "key", "value"],# Les couches spécifiques du ViT à modifier (Attention)
    lora_dropout=0.05,                       # Dropout appliqué aux couches LoRA
    bias="none",                             # Ne pas adapter les biais
    task_type=TaskType.FEATURE_EXTRACTION,   # Le ViT sert d'extracteur de features avant la tête de profondeur
)

In [17]:
# La fonction get_peft_model gèle les poids du modèle de base et ajoute les adaptateurs LoRA
model_lora = get_peft_model(base_model, lora_config)

print("✅ Modèle LoRA appliqué !")
# Afficher le nombre de paramètres entraînés
model_lora.print_trainable_parameters()

✅ Modèle LoRA appliqué !
trainable params: 442,368 || all params: 25,227,457 || trainable%: 1.7535


In [18]:
# C'est ici que l'extraction et le nettoyage à la volée se produisent
def __getitem__(self, idx):
    # ... chargement de l'image ...
    
    # 1. Chargement du fichier complet (lent)
    depth_data = np.load(depth_path) 
    
    # 2. Extraction du Z et nettoyage des NaN (ultra-rapide)
    z_channel = depth_data[:, :, 2]
    z_channel = np.nan_to_num(z_channel, nan=0.0) 
    
    # 3. Le processeur le prépare pour le GPU
    # ...
    return { ... }

In [19]:
# ===================================================
# CALCUL DES STATISTIQUES GLOBALES DU DATASET
# ===================================================
print("📊 Calcul des statistiques sur TOUS les fichiers depth...")
print("   (Cela peut prendre quelques minutes...)\n")

depth_stats = {
    'min_values': [],
    'max_values': [],
    'mean_values': [],
    'valid_percentages': []
}

# Parcourir TOUS les fichiers .npy
from tqdm import tqdm  # Pour une barre de progression

for depth_file in tqdm(files_npy, desc="Analyse des fichiers depth"):
    depth_path = os.path.join(path_depths, depth_file)
    
    # Charger et extraire Z
    depth_xyz = np.load(depth_path)
    depth_z = depth_xyz[:, :, 2]
    
    # Calculer statistiques (en ignorant NaN)
    if not np.all(np.isnan(depth_z)):  # Si au moins une valeur valide
        depth_stats['min_values'].append(np.nanmin(depth_z))
        depth_stats['max_values'].append(np.nanmax(depth_z))
        depth_stats['mean_values'].append(np.nanmean(depth_z))
        
        # % de pixels valides
        valid_pct = (~np.isnan(depth_z)).sum() / depth_z.size * 100
        depth_stats['valid_percentages'].append(valid_pct)

# Calculer les statistiques globales
GLOBAL_MIN = np.min(depth_stats['min_values'])
GLOBAL_MAX = np.max(depth_stats['max_values'])
GLOBAL_MEAN = np.mean(depth_stats['mean_values'])
GLOBAL_STD = np.std(depth_stats['mean_values'])

print("\n" + "="*60)
print("STATISTIQUES GLOBALES DU DATASET")
print("="*60)
print(f"📏 Profondeur MIN (sur tous les fichiers): {GLOBAL_MIN:.2f} mm")
print(f"📏 Profondeur MAX (sur tous les fichiers): {GLOBAL_MAX:.2f} mm")
print(f"📊 Moyenne des moyennes: {GLOBAL_MEAN:.2f} mm")
print(f"📊 Écart-type: {GLOBAL_STD:.2f} mm")
print(f"✅ Pixels valides (moyenne): {np.mean(depth_stats['valid_percentages']):.1f}%")
print("="*60)

# Ces valeurs vont servir pour la normalisation!
print(f"\n💡 Utilisez ces valeurs dans votre Dataset:")
print(f"   depth_min = {GLOBAL_MIN:.2f}")
print(f"   depth_max = {GLOBAL_MAX:.2f}")

📊 Calcul des statistiques sur TOUS les fichiers depth...
   (Cela peut prendre quelques minutes...)



Analyse des fichiers depth: 100%|██████████| 58/58 [00:17<00:00,  3.38it/s]


STATISTIQUES GLOBALES DU DATASET
📏 Profondeur MIN (sur tous les fichiers): 251.74 mm
📏 Profondeur MAX (sur tous les fichiers): 3907.45 mm
📊 Moyenne des moyennes: 1542.16 mm
📊 Écart-type: 295.35 mm
✅ Pixels valides (moyenne): 68.5%

💡 Utilisez ces valeurs dans votre Dataset:
   depth_min = 251.74
   depth_max = 3907.45


In [20]:
# ===================================================
# DATASET AVEC NORMALISATION CORRECTE
# ===================================================
from torch.utils.data import Dataset, DataLoader
class ZividDepthDataset(Dataset):
    """
    Dataset pour charger les paires RGB-Depth Zivid
    """
    def __init__(self, image_folder, depth_folder, image_files, depth_files, 
                 processor, depth_min, depth_max):
        """
        Args:
            image_folder: Chemin vers dossier images
            depth_folder: Chemin vers dossier depth
            image_files: Liste des noms de fichiers images
            depth_files: Liste des noms de fichiers depth
            processor: ImageProcessor de Hugging Face
            depth_min: Valeur minimale de profondeur (en mm)
            depth_max: Valeur maximale de profondeur (en mm)
        """
        self.image_folder = image_folder
        self.depth_folder = depth_folder
        self.image_files = image_files
        self.depth_files = depth_files
        self.processor = processor
        self.depth_min = depth_min
        self.depth_max = depth_max
        
        print(f"📊 Dataset créé:")
        print(f"   Échantillons: {len(self.image_files)}")
        print(f"   Normalisation: [{depth_min:.2f}, {depth_max:.2f}] mm → [0, 1]")
    
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        # 1. Charger l'image RGB
        img_path = os.path.join(self.image_folder, self.image_files[idx])
        image = Image.open(img_path).convert('RGB')
        
        # 2. Charger le fichier depth XYZ
        depth_path = os.path.join(self.depth_folder, self.depth_files[idx])
        depth_xyz = np.load(depth_path)
        
        # 3. Extraire le canal Z (profondeur)
        depth_z = depth_xyz[:, :, 2]  # Shape: (1200, 1944)
        
        # 4. Créer un masque des pixels valides (pour la loss)
        valid_mask = ~np.isnan(depth_z)
        
        # 5. Remplacer NaN par 0 temporairement
        depth_z_clean = np.nan_to_num(depth_z, nan=0.0)
        
        # 6. Normaliser la profondeur entre 0 et 1
        depth_normalized = (depth_z_clean - self.depth_min) / (self.depth_max - self.depth_min)
        depth_normalized = np.clip(depth_normalized, 0, 1)
        
        # 7. Preprocessing de l'image
        inputs = self.processor(images=image, return_tensors="pt")
        
        # 8. Retourner tout
        return {
            'pixel_values': inputs['pixel_values'].squeeze(0),
            'depth_gt': torch.from_numpy(depth_normalized).float(),
            'valid_mask': torch.from_numpy(valid_mask).float(),
            'filename': self.image_files[idx]
        }

print("✅ Classe Dataset (v2) définie avec normalisation!")

✅ Classe Dataset (v2) définie avec normalisation!


In [21]:
# ===================================================
# SPLIT TRAIN / VALIDATION
# ===================================================
from sklearn.model_selection import train_test_split

# Vos fichiers déjà triés
files_img = sorted([f for f in os.listdir(path_images) if f.endswith('.png')])
files_npy = sorted([f for f in os.listdir(path_depths) if f.endswith('.npy')])

print(f"📊 Total de fichiers: {len(files_img)}")

# Split 80% train, 20% validation
train_imgs, val_imgs, train_depths, val_depths = train_test_split(
    files_img, files_npy, 
    test_size=0.2,  # 20% validation
    random_state=42,  # Pour reproductibilité
    shuffle=True
)

print(f"✅ Split effectué:")
print(f"   Train: {len(train_imgs)} échantillons")
print(f"   Val:   {len(val_imgs)} échantillons")

📊 Total de fichiers: 58
✅ Split effectué:
   Train: 46 échantillons
   Val:   12 échantillons


In [22]:
# ===================================================
# CRÉATION DES DATALOADERS
# ===================================================

# Créer les datasets
train_dataset = ZividDepthDataset(
    image_folder=path_images,
    depth_folder=path_depths,
    image_files=train_imgs,
    depth_files=train_depths,
    processor=image_processor,
    depth_min = 251.74,
   depth_max = 3907.45
)

val_dataset = ZividDepthDataset(
    image_folder=path_images,
    depth_folder=path_depths,
    image_files=val_imgs,
    depth_files=val_depths,
    processor=image_processor,
    depth_min = 251.74,
   depth_max = 3907.45
)

# Créer les dataloaders
BATCH_SIZE = 2  # Petit batch car images volumineuses

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,  # Mélanger pour l'entraînement
    num_workers=0,  # 0 pour Windows, 2-4 sinon
    pin_memory=True if torch.cuda.is_available() else False
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,  # Pas de mélange pour validation
    num_workers=0,
    pin_memory=True if torch.cuda.is_available() else False
)

print("✅ DataLoaders créés!")
print(f"   Train batches: {len(train_loader)}")
print(f"   Val batches: {len(val_loader)}")

📊 Dataset créé:
   Échantillons: 46
   Normalisation: [251.74, 3907.45] mm → [0, 1]
📊 Dataset créé:
   Échantillons: 12
   Normalisation: [251.74, 3907.45] mm → [0, 1]
✅ DataLoaders créés!
   Train batches: 23
   Val batches: 6


In [23]:
# ===================================================
# TEST DU DATALOADER
# ===================================================
print("\n🧪 Test du DataLoader...")

# Récupérer un batch
batch = next(iter(train_loader))

print(f"✅ Batch chargé avec succès!")
print(f"   pixel_values shape: {batch['pixel_values'].shape}")
print(f"   depth_gt shape: {batch['depth_gt'].shape}")
print(f"   valid_mask shape: {batch['valid_mask'].shape}")
print(f"   Fichiers: {batch['filename']}")

# Vérifier les valeurs
print(f"\n📊 Statistiques:")
print(f"   Depth GT min: {batch['depth_gt'].min():.3f}")
print(f"   Depth GT max: {batch['depth_gt'].max():.3f}")
print(f"   Pixels valides: {batch['valid_mask'].sum().item()} / {batch['valid_mask'].numel()}")


🧪 Test du DataLoader...
✅ Batch chargé avec succès!
   pixel_values shape: torch.Size([2, 3, 518, 840])
   depth_gt shape: torch.Size([2, 1200, 1944])
   valid_mask shape: torch.Size([2, 1200, 1944])
   Fichiers: ['21-12-03-18-52-52_Zivid_acquisition_color.png', '21-12-03-18-52-40_Zivid_acquisition_color.png']

📊 Statistiques:
   Depth GT min: 0.000
   Depth GT max: 0.732
   Pixels valides: 3184454.0 / 4665600


In [24]:
# ==========================================
# 3. DÉFINITION DES ARGUMENTS ET DU TRAINER
# ==========================================
from transformers import IntervalStrategy,TrainingArguments, Trainer
print("\n3. Initialisation du Trainer...")
OUTPUT_DIR = "./depth_anything_finetuned_lora_zivid" 
BATCH_SIZE = 4 # ⚠️ IMPORTANT: Réduire cette valeur (2 ou 1) si votre GPU manque de VRAM

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    learning_rate=5e-5,               
    num_train_epochs=5,               
    per_device_train_batch_size=BATCH_SIZE,      
    per_device_eval_batch_size=BATCH_SIZE,              
    eval_steps=500,                                  
    save_steps=500,     
    # Reste inchangé :
    logging_steps=50,     
    report_to="tensorboard",          
    save_total_limit=3,               
    fp16=True,                        
    remove_unused_columns=False,        
)


3. Initialisation du Trainer...


In [46]:
%pip install tensorboard

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/5.5 MB ? eta -:--:--
   ---------------------------------------- 5.5/5.5 MB 49.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/4.7 MB ? eta -:--:--
   ---------------------------------------- 4.7/4.7 MB 87.8 MB/s eta 0:00:00

   ---------- ----------------------------- 1/4 [grpcio]
   ---------- ----------------------------- 1/4 [grpcio]
   ------------------------------ --------- 3/4 [tensorboard]
   ------------------------------ --------- 3/4 [tensorboard]
   ------------------------------ --------- 3/4 [tensorboard]
   ------------------------------ --------- 3/4 [tensorboard]
   ------------------------------ --------- 3/4 [tensorboard]
   ------------------------------ --------- 3/4 [tensorboard]
   ------------------------------ --------- 3/4 [tensorboard]
   ---------------------------------------- 4/4 [tensorboard]

Note: you may need

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [29]:
import torch
from torch.utils.data._utils.collate import default_collate

def depth_data_collator(features):
    """
    Utilise le collate par défaut de PyTorch après avoir s'être assuré
    que les features sont bien des dictionnaires avec les clés 'pixel_values' et 'labels'.
    """
    
    # Le collator par défaut gère l'empilement de tenseurs
    # On lui passe une liste de dictionnaires
    batch = default_collate(features)
    
    # On s'assure que seules les clés essentielles sont présentes (pixel_values et labels)
    # Dans les versions récentes, le collator par défaut de PyTorch/Hugging Face
    # peut inclure des clés non pertinentes. Pour simplifier, on prend juste ce dont on a besoin.
    
    return {
        "pixel_values": batch["pixel_values"],
        "labels": batch["labels"]
    }

print("✅ Data collator explicite mis à jour.")

✅ Data collator explicite mis à jour.


In [30]:
# Initialisation du Trainer (Corrigée)
trainer = Trainer(
    model=model_lora,  
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    #compute_metrics=compute_metrics,
    
    # 💥 Ajout de l'argument data_collator
    data_collator=depth_data_collator, 
)

print("\n🚀 Trainer initialisé avec le Data Collator corrigé.")

# LANCEMENT !
trainer.train()


🚀 Trainer initialisé avec le Data Collator corrigé.


KeyError: 'labels'